In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
from pathlib import Path

from google.cloud import aiplatform
from google.cloud.aiplatform import model_monitoring
from kfp.dsl import Model
from loguru import logger

module_path = os.path.abspath("..")
sys.path.append(module_path)

from src.base.utilities import read_json, read_yaml
from src.utils.notebooks import patch_kfp

patch_kfp()

from src.components.aiplatform import deploy_model, export_model, lookup_model

# Remove default logger and set level to DEBUG
logger.remove()
logger.add(sys.stderr, level="DEBUG")

In [ ]:
project_id = os.environ.get("VERTEX_PROJECT_ID")
project_location = os.environ.get("VERTEX_LOCATION")

In [ ]:
project_location

In [ ]:
monitoring_config = read_yaml("../src/pipelines/configuration/endpoint_monitoring.yaml")

In [ ]:
dataset_id = "credit_card_frauds"
data_version = "20230614145952"

In [ ]:
endpoint = aiplatform.Endpoint(
    "projects/386525277255/locations/europe-west2/endpoints/credit-card-frauds-endpoint"
)

In [ ]:
aiplatform.Model.list(project="robertofierimonte-ml-pipe", location="europe-west2")

In [ ]:
aiplatform.init(project="386525277255", location=project_location)
print(aiplatform.ModelDeploymentMonitoringJob.list())
print(aiplatform.Endpoint.list())
print(aiplatform.Model.list(project=project_id, location=project_location))

In [ ]:
aiplatform.ModelDeploymentMonitoringJob.list()[0].delete()

In [ ]:
skew_config = model_monitoring.SkewDetectionConfig(
    data_source=f"bq://{project_id}.{dataset_id}_{data_version}.training",  # noqa: E501
    skew_thresholds=monitoring_config["skew_thresholds"],
    attribute_skew_thresholds=monitoring_config["attribute_skew_thresholds"],
    target_field=monitoring_config["target_column"],
)
drift_config = model_monitoring.DriftDetectionConfig(
    drift_thresholds=monitoring_config["drift_thresholds"],
    attribute_drift_thresholds=monitoring_config["attribute_drift_thresholds"],
)
# explanation_config = model_monitoring.ExplanationConfig()
objective_config = model_monitoring.ObjectiveConfig(
    skew_detection_config=skew_config,
    drift_detection_config=drift_config,
    # explanation_config=explanation_config,
)

# Create sampling configuration.
sampling_config = model_monitoring.RandomSampleConfig(sample_rate=1.0)

# Create schedule configuration.
schedule_config = model_monitoring.ScheduleConfig(monitor_interval=6)

# Create alerting configuration.
alerting_config = model_monitoring.EmailAlertConfig(
    user_emails=["roberto.fierimonte@gmail.com"], enable_logging=True
)

# Create the monitoring job.
job = aiplatform.ModelDeploymentMonitoringJob.create(
    display_name=f"monitoring-job-credit-card-frauds-endpoint",
    logging_sampling_strategy=sampling_config,
    schedule_config=schedule_config,
    alert_config=alerting_config,
    objective_configs=objective_config,
    project=project_id,
    location=project_location,
    endpoint=endpoint
)

In [ ]:
aiplatform.init(project=project_id, location=project_location)

In [ ]:
aiplatform.ModelDeploymentMonitoringJob(
    "projects/386525277255/locations/europe-west2/modelDeploymentMonitoringJobs/456",
)